**Sars-Cov-2 identificator**

This model was trained to identify whether a person might be infected by Sars-Cov-2 or just a simple illness.

## Run this model

1. Install the requirements.txt

`pip install -r requirements.txt`




In [1]:
import pickle
import pandas as pd
from sklearn.naive_bayes import GaussianNB
import sqlite3
from training_data import data

In [18]:
pip freeze > requirements.txt

In [2]:
cnx = sqlite3.connect('covid_data.sqlite')
cur = cnx.cursor()

In [3]:
# Creating a table to keep patient informations
cur.execute("""
  CREATE TABLE IF NOT EXISTS patient_data (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    fever INTEGER NOT NULL,
    fatigue INTEGER NOT NULL,
    cough INTEGER NOT NULL,
    sneezing INTEGER NOT NULL,
    body_aches INTEGER NOT NULL,
    runny_nose INTEGER NOT NULL,
    sore_throat INTEGER NOT NULL,
    diarrhea INTEGER NOT NULL,
    headache INTEGER NOT NULL,
    shortness_of_breath INTEGER NOT NULL,
    result INTEGER NOT NULL
  )
""")

In [4]:
for i, atributos in enumerate(data['atributos']):
  cur.execute("""
  INSERT INTO patient_data (fever, fatigue, cough, sneezing, body_aches, runny_nose, sore_throat, diarrhea, headache, shortness_of_breath, result)
  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
  """, atributos + [data['resultados'][i]])

In [5]:
cnx.commit()

In [6]:
# Com o Pandas, iniciar a recuperação de informações
df = pd.read_sql_query('SELECT * FROM patient_data', cnx)

In [7]:
# Closing database connection
cnx.close()

In [8]:
# Removing id column from df
df = df.drop('id', axis=1)

In [9]:
params_columns = df[df.columns[:-1]]
params = params_columns.to_numpy()

In [10]:
# Iniciando o fit com GaussianNB
ia = GaussianNB()
ia.fit(params, df['result'].to_numpy())

GaussianNB()

In [17]:
# Saving trained model
# with open('model.pkl', 'wb') as file:
#   pickle.dump(ia, file)

# Opening trained model
# loaded_model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
patient_answers = []
for i in range(10):
  answer = int(input(data['questions'][i]))
  patient_answers.append(answer)

In [16]:
# Predicting according to answers
hasCovid = ia.predict([patient_answers])
print('Recomenda-se fazer o teste de covid' if hasCovid[0] == 1 else 'Paciente não apresenta sintomas de covid')

Paciente não apresenta sintomas de covid
